# Team Grades
In this notebook we will look at team-related PFF grades and their impact on individual player fantasy production.

#### __Goal__: Discover how team-related performance (blocking, defense, etc.) affects each position's fantasy performance.

This analysis will specifically look at __PPR__ scoring.

The data only contains full-season data (not game-by-game), so it is tailored towards __redraft__ fantasy football leagues. It may not be as predictive for best ball leagues.

In [4]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# display
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

# global random_state
random_state = 9

In [32]:
def view_player(player, df):
    return df[df['Player'] == player]

We will load in the 4 positional subsets. Since we don't have PFF grades prior to 2006, we will drop these seasons.

In [17]:
# load data
qb = pd.read_csv('../data/final_data/qb.csv')
rb = pd.read_csv('../data/final_data/rb.csv')
wr = pd.read_csv('../data/final_data/wr.csv')
te = pd.read_csv('../data/final_data/te.csv')

# drop seasons before 2006
qb = qb[qb['Year'] >= 2006]
rb = rb[rb['Year'] >= 2006]
wr = wr[wr['Year'] >= 2006]
te = te[te['Year'] >= 2006]

# check shapes
qb.shape, rb.shape, wr.shape, te.shape

((1398, 172), (3187, 173), (4094, 161), (2243, 161))

Many of the seasons that we have are seasons of backup players with very little volume. The goal of this analysis is to identify how much __team performance__ impacts a fantasy player's performance. We will identify all players who have finished at or above replacement-level (in PPG) at least 1 time in their career. This will be the subset that we will analyze.

In [18]:
# List of DataFrames to iterate over
dfs = [qb, rb, wr, te]

# Iterate over each position and update the DataFrame
for i in range(len(dfs)):
    df = dfs[i]
    
    # Get all unique 'Key' values that have at least 1 'PPG_VORP_half-ppr_12tm_3WR' value >= 0
    keys = df[df['PPG_VORP_half-ppr_12tm_3WR'] >= 0]['Key'].unique()

    # Update the DataFrame to drop all rows with 'Key' values not in the key set
    dfs[i] = df[df['Key'].isin(keys)]

# Assign the updated DataFrames back to the original variables
qb, rb, wr, te = dfs

# check shapes
qb.shape, rb.shape, wr.shape, te.shape

((615, 172), (1163, 173), (1569, 161), (601, 161))

## QB

In [29]:
# create '%Pts_rush' col for QBs
qb['%Pts_rush'] = ((qb['Rush_Yds'] * 0.1) + (qb['Rush_TD'] * 6)) / qb['Points_standard']

# inspect new col
qb['%Pts_rush'].describe()

# define a 'rushing' QB threshold

count    610.000000
mean       0.141060
std        0.357379
min       -5.000000
25%        0.036948
50%        0.097417
75%        0.197577
max        2.674419
Name: %Pts_rush, dtype: float64

In [35]:
view_player('Lamar Jackson', qb)[['Year', 'PPG_VORP_half-ppr_12tm_3WR', '%Pts_rush']]

,Year,PPG_VORP_half-ppr_12tm_3WR,%Pts_rush
3713,2018,-8.369464,0.631586
3795,2019,10.052000,0.391166
3872,2020,0.935750,0.430800
3961,2021,0.893750,0.375879
4042,2022,2.004848,0.403281
4124,2023,2.870096,0.340502


In [36]:
view_player('Josh Allen', qb)[['Year', 'PPG_VORP_half-ppr_12tm_3WR', '%Pts_rush']]

,Year,PPG_VORP_half-ppr_12tm_3WR,%Pts_rush
3705,2018,-1.044048,0.539163
3801,2019,0.000000,0.371602
3873,2020,3.606250,0.228067
3952,2021,4.556985,0.283171
4036,2022,6.968182,0.301900
4122,2023,5.037376,0.368301


In [39]:
view_player('Jalen Hurts', qb)[['Year', 'PPG_VORP_half-ppr_12tm_3WR', '%Pts_rush']]

,Year,PPG_VORP_half-ppr_12tm_3WR,%Pts_rush
3864,2020,-13.873583,0.493804
3948,2021,1.639417,0.452051
4037,2022,7.434182,0.411721
4123,2023,3.283258,0.421781


In [38]:
view_player('Patrick Mahomes', qb)[['Year', 'PPG_VORP_half-ppr_12tm_3WR', '%Pts_rush']]

,Year,PPG_VORP_half-ppr_12tm_3WR,%Pts_rush
3596,2017,-6.492500,0.096525
3724,2018,7.726786,0.094440
3802,2019,2.700000,0.118580
3876,2020,3.443750,0.116178
3956,2021,2.267574,0.140077
4035,2022,6.668770,0.145712
4131,2023,-0.192404,0.138819


### Receiving
How does a team's receiving grade affect QB fantasy performance?

In [42]:
# look at cols that contain 'Team'
team_cols = qb.columns[qb.columns.str.contains('Team')].to_list()

# see how team_cols correlate with 'PPG_half-ppr'
qb[team_cols + ['PPG_half-ppr']].corr()[['PPG_half-ppr']].sort_values(by='PPG_half-ppr', ascending=False).T

,PPG_half-ppr,Team_PPG,Team_Passing Grade,Team_Receiving Grade,Team_Offense Grade,Team_Wins,Team_Team Grade,Team_Rushing Grade,Team_Special Teams Grade,Team_PPG_allowed,Team_Pass Rush Grade,Team_Pass Blocking Grade,Team_Run Blocking Grade,Team_Coverage Grade,Team_Defense Grade,Team_Run Defense Grade,Team_Tackling Grade,Team_Losses,Will_be_on_New_Team,New_Team,Rush_Team_Rush%
PPG_half-ppr,1.0,0.331125,0.330024,0.284662,0.277351,0.197201,0.186525,0.177118,0.03291,0.029373,-0.012986,-0.025557,-0.032754,-0.034064,-0.071875,-0.092993,-0.120411,-0.19398,-0.219034,-0.292633,-0.315097


In [ ]:
# drop null targets
corr_df = top_25.dropna(subset=['SeasonTarget_ppr'])

# get correlation with Season Target
corr = corr_df[team_cols].corr()[['SeasonTarget_ppr']].sort_values(by='SeasonTarget_ppr', ascending=False).T